In [1]:
2+2

4

In [2]:
import os
import json

In [3]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list

# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561197996873645
Review: Awesome game, but I held off playing it because of the top-down/isometric camera - didn't like the camera in DOS2 either.  Then I found the "Native Camera Tweaks" mod on Nexus Mods.

This totally changes the game, allowing you to use standard 3rd person camera controls, and actually see what's ahead when moving up-hill.

It turns out all outdoor areas have sky boxes (including sun and moon), and almost all interior areas have ceilings.  Trust me, you'll hardly notice the few times a ceiling is missing or "pops in", unless you're looking for it.  And there's no performance hit that I've noticed.

Makes me wander why Larian didn't give an unlocked camera by default...

Also you have full control over zoom/pitch/FoV/invert axis/dead zones etc through the mod's configuration file.

[b][i]EDIT:  The mod has been updated (8 SEP 24) to work with patch 7.  Many thanks to Ersh for updating it so promptly!
Timestamp Created: 1724579126
-----------------------

In [4]:
import os
import json
import random  # Import random for shuffling

# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            
            # If the loaded content is a list of reviews, extend the main list
            if isinstance(file_reviews, list):
                reviews.extend(file_reviews)  # Append reviews to the main list
            else:
                reviews.append(file_reviews)  # Handle the case where it's not a list

# Shuffle the reviews to randomize their order
random.shuffle(reviews)

# Print the first i shuffled reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)


Author ID: 76561197970791943
Review: BG3 is my favorite game of all time if that means anything to you. Probably not since you don't know me. Hi, my name is Warna. Now that you know me better, this is my favorite game of all time.
Timestamp Created: 1726607144
-------------------------------------------------------------------------------
Author ID: 76561198096877172
Review: Positive:
This Game Is amazing It runs smooth as Silk for me 170 FPS, Ultra Settings NVIDIA DLSS Set to Ultra Performance

Negative:
I cannot wrap my head around this, The PSN account link. Why put that In If you don't get anything back In return?

To The Negative Fans:
You don't need to have PSN Linkage to play the story mode I haven't linked mine and I've just beat Thor In Midgard so stop stressing
Timestamp Created: 1726773722
-------------------------------------------------------------------------------


In [6]:
from openai import OpenAI

client = OpenAI()

In [7]:
prompt_template = """
You are emulating a user of our computer game preselection application.
The application and its recommendations are based on reviews from an online shop called Steam.
The application is designed for conservative fathers of young children who are not aware of how the modern gaming industry works.
This application is for fathers who are not up to date with the latest titles released on a daily basis.
Formulate 5 questions this user who is a father might ask based on a provided exercise.
Make the questions specific to this exercise and user, who is a father. Remeber, user needs to keep his child from not suitable games, including DEI games.
DEI ideology, which enforces Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda to improve sales. 
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

title: {title}
language: {language}
review: {review}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [8]:
prompt = prompt_template.format(**reviews[0])

In [9]:
prompt

'You are emulating a user of our computer game preselection application.\nThe application and its recommendations are based on reviews from an online shop called Steam.\nThe application is designed for conservative fathers of young children who are not aware of how the modern gaming industry works.\nThis application is for fathers who are not up to date with the latest titles released on a daily basis.\nFormulate 5 questions this user who is a father might ask based on a provided exercise.\nMake the questions specific to this exercise and user, who is a father. Remeber, user needs to keep his child from not suitable games, including DEI games.\nDEI ideollogy, which stands fro Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda. \nThe record should contain the answer to the questions, and the questions should\nbe complete and not too short. Use as fewer words as possible from the record. \n\nThe record:\n\ntitle: Baldur\'s Gate 3\nlanguage: english\nre

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [11]:
questions = llm(prompt)

In [12]:
json.loads(questions)

{'questions': ["What are the main themes and content of Baldur's Gate 3, and are they suitable for young children?",
  "Can you provide information on the game's visual style and how that might affect a child's perception?",
  "Are there any elements in Baldur's Gate 3 that promote diversity or inclusion, and how do they impact the overall gaming experience?",
  "Is there any violence or mature content in Baldur's Gate 3 that I should be aware of before letting my child play?",
  "Do you have any recommendations for games similar to Baldur's Gate 3 that are more suitable for kids?"]}

In [13]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [14]:
from tqdm.auto import tqdm

In [15]:
results = {}

In [16]:
for doc in tqdm(reviews): 
    doc_id = doc['appid']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/200 [00:00<?, ?it/s]

In [17]:
final_results = []

for doc_id, reviews in results.items():
    for q in reviews:
        final_results.append((doc_id, q))

In [18]:
final_results[0]

('1086940',
 "What makes Baldur's Gate 3 a suitable choice for my child, considering I want to avoid games influenced by DEI ideologies?")

In [19]:
final_results[-1]

('2322010',
 'What are some recommended parental controls or settings I should consider while my child is playing games like God of War: Ragnarok?')

In [20]:
import json
import os

# Assuming final_results is defined as a list of dictionaries
# Example: final_results = [{'recommendationid': 1, 'question': 'What is the best game?'}, ...]
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground-truth-retrieval.json")

# Save the results to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(final_results, json_file, ensure_ascii=False, indent=4)

print(f"Data saved to {output_file}")


Data saved to /mnt/c/Users/KonuTech/llm-zoomcamp-capstone-01/reviews-assistant/data/ground_truth/ground-truth-retrieval.json
